In [10]:
import numpy as np
from tqdm import tqdm

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [11]:
def get_w_d(geno):
    Aff = np.ones((7, 7)) * -1 # from x to
    np.fill_diagonal(Aff, 0)
    Aff[2:6, -1] = 0
    for edge_idx, (op, in_node) in enumerate(geno):
        Aff[in_node, 2 + edge_idx//2] = edge2value(op)  # start from node #2, two edges per node
    depth, width = effective_depth_width(Aff)
    return depth, width

In [17]:
def edge2value(op_str):
    if 'skip' in op_str or 'pool' in op_str: return 0
    if 'conv' in op_str: return 1
    else: return -1

def find_all_paths(Aff, all_paths, curr_path=[], curr_pos=0, end_pos=5):
    if curr_pos == end_pos:
        all_paths.append(list(curr_path))
        return

    next_nodes = np.where(Aff[curr_pos, (curr_pos+1):] >= 0)[0] + curr_pos + 1
    for node in next_nodes:
        curr_path.append(Aff[curr_pos, node])
        find_all_paths(Aff, all_paths, curr_path, node, end_pos)
        curr_path.pop(-1)
    return all_paths


def effective_depth_width(Aff):
    Aff0 = np.array(Aff)
    Aff0 = np.delete(Aff0, 1, 0)
    Aff0 = np.delete(Aff0, 1, 1)
    paths0 = find_all_paths(Aff0, [])
    paths1 = find_all_paths(Aff[1:, 1:], [])
    paths = paths0 + paths1
    depth = 0
    width = 0
    for path in paths:
        depth += np.sum(path)
        width += int(np.sum(path) > 0)
    if depth == 0: return 0, 0
    else:
        depth = depth / len(paths)
        return depth, width/depth

In [12]:
def rand_arch_darts():
    OPS = [
        'max_pool_3x3',
        'avg_pool_3x3',
        'skip_connect',
        'sep_conv_3x3',
        'sep_conv_5x5',
        'dil_conv_3x3',
        'dil_conv_5x5'
       ]
    normal = [['dil_conv_5x5', 0], ['sep_conv_5x5', 1], ['dil_conv_5x5', 0], ['sep_conv_5x5', 1], ['dil_conv_5x5', 0], ['max_pool_3x3', 3], ['sep_conv_3x3', 4], ['dil_conv_5x5', 0]]
    from_node_1 = [[0, 1], [0, 2], [1, 2]]
    from_node_2 = [[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]
    from_node_3 = [[0, 1], [0, 2], [0, 3], [0, 4], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]
    for edge in range(8):
        normal[edge][0] = np.random.choice(OPS)
    idx = np.random.choice(len(from_node_1))
    normal[2][1] = from_node_1[idx][0]
    normal[3][1] = from_node_1[idx][1]
    idx = np.random.choice(len(from_node_2))
    normal[4][1] = from_node_2[idx][0]
    normal[5][1] = from_node_2[idx][1]
    idx = np.random.choice(len(from_node_3))
    normal[6][1] = from_node_3[idx][0]
    normal[7][1] = from_node_3[idx][1]
    return normal


In [15]:
import os
from collections import namedtuple

from ConfigSpace.read_and_write import json as cs_json

import nasbench301 as nb

# Default dirs for models
# Note: Uses 0.9 as the default models, switch to 1.0 to use 1.0 models
version = '0.9'

# current_dir = os.path.dirname(os.path.abspath(__file__))
current_dir = os.path.dirname(os.path.abspath(''))
models_0_9_dir = os.path.join(current_dir, 'nb_models_0.9')
model_paths_0_9 = {
    model_name : os.path.join(models_0_9_dir, '{}_v0.9'.format(model_name))
    for model_name in ['xgb', 'gnn_gin', 'lgb_runtime']
}
models_1_0_dir = os.path.join(current_dir, 'nb_models_1.0')
model_paths_1_0 = {
    model_name : os.path.join(models_1_0_dir, '{}_v1.0'.format(model_name))
    for model_name in ['xgb', 'gnn_gin', 'lgb_runtime']
}
model_paths = model_paths_0_9 if version == '0.9' else model_paths_1_0

# If the models are not available at the paths, automatically download
# the models
# Note: If you would like to provide your own model locations, comment this out
if not all(os.path.exists(model) for model in model_paths.values()):
    nb.download_models(version=version, delete_zip=True,
                       download_dir=current_dir)

# Load the performance surrogate model
#NOTE: Loading the ensemble will set the seed to the same as used during training (logged in the model_configs.json)
#NOTE: Defaults to using the default model download path
print("==> Loading performance surrogate model...")
ensemble_dir_performance = model_paths['xgb']
print(ensemble_dir_performance)
performance_model = nb.load_ensemble(ensemble_dir_performance)

# Load the runtime surrogate model
#NOTE: Defaults to using the default model download path
print("==> Loading runtime surrogate model...")
ensemble_dir_runtime = model_paths['lgb_runtime']
runtime_model = nb.load_ensemble(ensemble_dir_runtime)

print("==> Creating test configs...")
Genotype = namedtuple('Genotype', 'normal normal_concat reduce reduce_concat')

1.69GiB [00:55, 30.5MiB/s]


Extracting models_0.9.zip to /home/xinyu_gong/project/nb_models_0.9
Deleting downloaded zip at /home/xinyu_gong/project/models_0.9.zip
==> Loading performance surrogate model...
/home/xinyu_gong/project/nb_models_0.9/xgb_v0.9
==> Loading runtime surrogate model...
==> Creating test configs...


In [ ]:
# depths = []
# widths = []
genos = []
num_samples = 10000
arch_info = {}
for _n in tqdm(range(num_samples)):
    # random select normal / reduce
    geno = [rand_arch_darts(), rand_arch_darts()]
    while geno in genos:
        geno = [rand_arch_darts(), rand_arch_darts()]
    genos.append(geno)
    # calculate depth / width for normal / reduce
    normal_cell_geno, reduce_cell_geno = geno
    normal_depth, normal_width = get_w_d(normal_cell_geno)
    reduce_depth, reduce_width = get_w_d(reduce_cell_geno)
    genotype_config = Genotype(
        normal=normal_cell_geno,
        normal_concat=[2, 3, 4, 5],
        reduce=reduce_cell_geno,
        reduce_concat=[2, 3, 4, 5]
        )
    prediction_genotype = performance_model.predict(config=genotype_config, representation="genotype", with_noise=False)
    runtime_genotype = runtime_model.predict(config=genotype_config, representation="genotype", with_noise=False)
    tqdm.write("Genotype architecture performance: %f, runtime %f" %(prediction_genotype, runtime_genotype))
    arch_info[_n] = {
        "normal_depth": normal_depth,
        "normal_width": normal_width,
        "reduce_depth": reduce_depth,
        "reduce_width": reduce_width,
        "perf": prediction_genotype,
        "run_time": runtime_genotype,
        "geno": geno,
    }
    
#     Aff = np.ones((7, 7)) * -1 # from x to
#     np.fill_diagonal(Aff, 0)
#     Aff[2:6, -1] = 0
#     for edge_idx, (op, in_node) in enumerate(geno):
#         Aff[in_node, 2 + edge_idx//2] = edge2value(op)  # start from node #2, two edges per node
#     depth, width = effective_depth_width(Aff)
#     depths.append(depth)
#     widths.append(width)

  0%|          | 1/10000 [00:01<2:51:09,  1.03s/it]

Genotype architecture performance: 93.457184, runtime 4184.201616


  0%|          | 2/10000 [00:01<2:21:42,  1.18it/s]

Genotype architecture performance: 92.230293, runtime 4208.419497


  0%|          | 3/10000 [00:02<2:21:46,  1.18it/s]

...

Genotype architecture performance: 93.407448, runtime 4652.690896


 50%|████▉     | 4956/10000 [1:17:04<1:11:32,  1.17it/s]

Genotype architecture performance: 92.926865, runtime 5435.065296


 50%|████▉     | 4957/10000 [1:17:05<1:10:09,  1.20it/s]

Genotype architecture performance: 92.542778, runtime 4392.217922


In [24]:
arch_info.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [25]:
arch_info[0]

{'normal_depth': 0.4,
 'normal_width': 10.0,
 'reduce_depth': 0.6,
 'reduce_width': 10.0,
 'perf': 93.457184,
 'run_time': 4184.2016160914645,
 'geno': [[['avg_pool_3x3', 0],
   ['avg_pool_3x3', 1],
   ['dil_conv_3x3', 0],
   ['skip_connect', 2],
   ['sep_conv_5x5', 0],
   ['skip_connect', 1],
   ['sep_conv_3x3', 0],
   ['skip_connect', 4]],
  [['sep_conv_3x3', 0],
   ['sep_conv_3x3', 1],
   ['skip_connect', 0],
   ['avg_pool_3x3', 1],
   ['max_pool_3x3', 0],
   ['skip_connect', 1],
   ['dil_conv_3x3', 3],
   ['dil_conv_5x5', 4]]]}

In [26]:
from copy import deepcopy
arch_info_more = deepcopy(arch_info)

In [27]:
for k, v in arch_info_more.items():
    normal_conv_geno, reduce_conv_geno = v["geno"]
    arch_info_more[k].update({
        "normal_conv_count": str(normal_conv_geno).count('conv'),
        "reduce_conv_count": str(reduce_conv_geno).count('conv'),
    })

In [35]:
for k, v in arch_info_more.items():
    for sub_k, sub_v in v.items():
        if isinstance(sub_v, np.float32):
            arch_info_more[k][sub_k] = float(sub_v)

In [36]:
import json
with open("nas301_preds.json", "w") as f:
    json.dump(arch_info_more, f)

In [37]:
arch_info_more[0]

{'normal_depth': 0.4,
 'normal_width': 10.0,
 'reduce_depth': 0.6,
 'reduce_width': 10.0,
 'perf': 93.45718383789062,
 'run_time': 4184.2016160914645,
 'geno': [[['avg_pool_3x3', 0],
   ['avg_pool_3x3', 1],
   ['dil_conv_3x3', 0],
   ['skip_connect', 2],
   ['sep_conv_5x5', 0],
   ['skip_connect', 1],
   ['sep_conv_3x3', 0],
   ['skip_connect', 4]],
  [['sep_conv_3x3', 0],
   ['sep_conv_3x3', 1],
   ['skip_connect', 0],
   ['avg_pool_3x3', 1],
   ['max_pool_3x3', 0],
   ['skip_connect', 1],
   ['dil_conv_3x3', 3],
   ['dil_conv_5x5', 4]]],
 'normal_conv_count': 3,
 'reduce_conv_count': 4}